In [5]:
# Implementing libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
os.getcwd()

'/content'

In [8]:
os.chdir("/content/drive/MyDrive/Netflix-Dataset/")

In [9]:
os.getcwd()

'/content/drive/MyDrive/Netflix-Dataset'